In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
using LinearAlgebra, Plots
import FiniteDiff
import ForwardDiff as FD
using MeshCat
using Test
using Plots

  Activating project at `c:\Users\aphia\Desktop\school\ocrl\QuadraCat`


In [2]:
# include the functions from quadruped.jl
include(joinpath(@__DIR__, "utils", "fmincon.jl"))
include(joinpath(@__DIR__, "utils", "quadruped.jl"))

# this loads in our continuous time dynamics function xdot = dynamics(model, x, u)

initialize_visualizer (generic function with 1 method)

In [3]:
model = UnitreeA1() # contains all the model properties for the quadruped

# tf = 5
# dt = 0.1

# t_vec = 0:dt:tf 
# N = length(t_vec)
# X = [zeros(state_dim(model)) for i in 1:N]

UnitreeA1{StateCache{Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}, Vector{Joint{Float64, Fixed{Float64}}}}, 2}}}(Spanning tree:
Vertex: world (root)
  Vertex: dummy1_RR, Edge: foot_joint_x_RR
    Vertex: dummy2_RR, Edge: foot_joint_y_RR
      Vertex: RR_foot, Edge: foot_joint_z_RR
        Vertex: RR_calf, Edge: RR_foot_fixed
          Vertex: RR_thigh, Edge: RR_calf_joint
            Vertex: RR_hip, Edge: RR_thigh_joint
              Vertex: trunk, Edge: RR_hip_joint
                Vertex: imu_link, Edge: imu_joint
                Vertex: FR_hip, Edge: FR_hip_joint
                  Vertex: FR_thigh, Edge: FR_thigh_joint
                    Vertex: FR_calf, Edge: FR_calf_joint
                      Vertex: FR_foot, Edge: FR_foot_fixed
                Vertex: FL_hip, Edge: FL_hip_joint
                  Vertex: FL_thigh, Edge: FL_thigh_joint
                    Vertex: FL_calf, Edge: FL_calf_joint
                      Vertex: FL_f

In [4]:
## set up discrete time dynamics

function stance_dynamics(params::NamedTuple, x::Vector, u::Vector)::Vector
    model = params.model
    return dynamics(model, x, u)
end

function flight_dynamics(params::NamedTuple, x::Vector, u::Vector)::Vector
    g = params.g
    v = x[16:end]

    v̇ = [0; 0; -g; zeros(12)]

    return [v; v̇]
end

function hermite_simpson_ground(params::NamedTuple, x1::Vector, x2::Vector, u::Vector, dt::Real)::Vector
    model = params.model
    x12 = 0.5*(x1+x2) + dt/8*(dynamics(model, x1, u) - dynamics(model, x2, u))
    return x1 + dt/6 * (dynamics(model, x1, u) + 4 * dynamics(model, x12, u) + dynamics(model, x2, u)) - x2
end

function hermite_simpson_flight(params::NamedTuple, x1::Vector, x2::Vector, u::Vector, dt::Real)::Vector
    x12 = 0.5*(x1+x2) + dt/8*(flight_dynamics(params, x1, u) - flight_dynamics(params, x2, u))
    return x1 + dt/6 * (flight_dynamics(params, x1, u) + 4 * flight_dynamics(params, x12, u) 
    + flight_dynamics(params, x2, u)) - x2
end

function jump_map(x::Vector)::Vector 
    # quadracat leaves the ground 
    # maintain joint and body positon, body velocity, zero joint velocities
    xn = [x[1:18]; zeros(12)]
    return xn
end

function rk4(params::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(params, x,        u)
    k2 = dt * ode(params, x + k1/2, u)
    k3 = dt * ode(params, x + k2/2, u)
    k4 = dt * ode(params, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end  

rk4 (generic function with 1 method)

In [5]:
function get_foot_position(params::NamedTuple, x::Vector, foot="RR")
    model = params.model
    mech = params.model.mech 
    T = eltype(x)
    state = MechanismState{T}(mech)
    set_configuration!(state, x[1:state_dim(model)÷2])

    world = root_body(mech) # world = findbody(mech, "world")
    foot_body = findbody(mech, foot * "_foot")
    tf_world = relative_transform(state, default_frame(world), default_frame(foot_body))
    foot_pos_world = translation(tf_world)
    return foot_pos_world
end

get_foot_position (generic function with 2 methods)

In [6]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [7]:
## set up cost

function quadracat_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf

    J = 0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 0.5*(xi - xg)' * Q * (xi - xg) + 0.5*ui' * R * ui

    end

    xn = Z[idx.x[N]]
    J += 0.5 * (xn - xg)' * Qf * (xn - xg)

    return J

end

quadracat_cost (generic function with 1 method)

Which goes into the following full optimization problem:
$$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & J(x_{1:N},u_{1:N-1}) & \\ 
\text{st} \quad &  x_1 = x_{ic} & \tag{1}\\
\quad & x_N = x_{g} &\tag{2}\\
&  x_{k+1} = f_1(x_k,u_k)  &\text{for } k \in  \mathcal{M}_1 \setminus \mathcal{J}_1 \tag{3}\\ 
&  x_{k+1} = f_2(x_k,u_k)  &\text{for } k \in  \mathcal{M}_2 \setminus \mathcal{J}_2 \tag{4}\\ 
&  x_{k+1} = g_2(f_1(x_k,u_k))  &\text{for } k \in   \mathcal{J}_1 \tag{5}\\ 
&  foot position_z = 0  &\text{for } k \in   \mathcal{M}_1 \tag{6}\\ 
&  x_{k}[3] \geq 0 & \text{for } k \in [1, N]\tag{7}
\end{align}$$

Each constraint is now described, with the type of constraint for `fmincon` in parantheses:
1. Initial condition constraint **(equality constraint)**. 
2. Terminal condition constraint **(equality constraint)**. 
3. Stance 1 discrete dynamics **(equality constraint)**. 
4. Stance 2 discrete dynamics **(equality constraint)**. 
5. Discrete dynamics from stance 1 to stance 2 with jump map **(equality constraint)**. 
6. Make sure all 4 feet are pinned to the ground in stance 1 **(equality constraint)**.
7. Keep the z position of all 3 bodies above ground **(primal bound)**.

In [8]:
## set up equality constraints 

function quadracat_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
    J1 = params.J1
    model = params.model 

    c = zeros(eltype(Z), idx.nc)

    for i=1:(N-1)
        xk = Z[idx.x[i]]
        uk = Z[idx.u[i]]
        xk1 = Z[idx.x[i+1]]
        if (i in M1) && !(i in J1) # stance 1 discrete dynamics
            c[idx.c[i]] = xk1-rk4(params, stance_dynamics, xk, uk, dt)
        elseif (i in M2) # stance 2 discrete dynamics
            c[idx.c[i]] = xk1-rk4(params, flight_dynamics, xk, uk, dt)
        elseif (i in J1) # jump map from stance 1 to stance 2
            c[idx.c[i]] = xk1-jump_map(rk4(params, stance_dynamics, xk, uk, dt))
        end
    end
    return c 
end

function quadracat_stance_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt, model = params.idx, params.N, params.dt, params.model
    M1, M2 = params.M1, params.M2 
    J1 = params.J1
    
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), length(M1)*4)
    
    # TODO: add stance constraints (constraints 6 in the opti problem)
    for i=1:length(M1)
        xk = Z[idx.x[M1[i]]]
        q = xk[1:state_dim(model)÷2]
        c[4*(i-1) .+ (1:4)] = [
                            get_foot_position(params, xk, "RR")[3]; 
                            get_foot_position(params, xk, "RL")[3];
                            get_foot_position(params, xk, "FL")[3];
                            get_foot_position(params, xk, "FR")[3]
                            ]
    end

    return c

end

function quadracat_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    # TODO: stack up all of our equality constraints 
    
    # should be length 2*nx + (N-1)*nx + N 
    # inital condition constraint (nx)       (constraint 1)
    # terminal constraint         (nx)       (constraint 2)
    # dynamics constraints        (N-1)*nx   (constraint 3-6)
    # stance constraint           N          (constraint 7-8)
    ic_con = Z[idx.x[1]] - xic
    ter_con = Z[idx.x[N]] - xg
    return [ic_con; 
    #ter_con; 
    quadracat_dynamics_constraints(params, Z); 
    quadracat_stance_constraint(params, Z)
    ]
end

quadracat_equality_constraint (generic function with 1 method)

In [15]:
## set up inequality constraint 

function quadracat_inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    xg = params.xg
        
    # create c in a ForwardDiff friendly way (check HW0)
    # c = zeros(eltype(Z), 1)
    c = [norm(Z[idx.x[N]][3] - xg[3])]

    return c
end

quadracat_inequality_constraint (generic function with 1 method)

In [10]:
## reference trajectory (straight line)

function reference_trajectory(params::NamedTuple, xic::Vector, xg::Vector)
    N = params.N
    Uref = [0.001*randn(params.nu) for i in 1:(N-1)]

    Xref = range(xic, xg, N)

    return Xref, Uref
end

reference_trajectory (generic function with 1 method)

In [13]:
function solve_quadracat_jump(;verbose=true)
    
    # problem size 
    nx = state_dim(model)
    nu = control_dim(model)
    # placeholder values for now. might need to tweak
    tf = 1.5
    dt = 0.15
    
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # LQR cost 
    Q = diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial and goal states 
    xic = initial_state(model)
    xg = deepcopy(xic)
    xg[3] = 0.5

    # index sets 
    t_jump = 1.0
    J1 = [findfirst(t -> t ≥ t_jump, t_vec)]
    M1 = collect(1:J1[1])
    M2 = collect((J1[1]+1):N)

    # load all useful things into params 
    params = (
        model=model,
        nx=nx, 
        nu=nu, 
        Q = Q, 
        R = R, 
        Qf = Qf, 
        xic = xic, 
        xg = xg, 
        dt = dt, 
        N = N, 
        idx = idx,
        J1 = J1,
        M1 = M1,
        M2 = M2,
        g = 9.81,
        )
    
    # TODO: primal bounds (constraint 9)
    x_l = -Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        x_l[idx.x[i][3]] = 0
    end
    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(1) #zeros(0)
    c_u = [1e-2]
    # function inequality_constraint(params, Z)
    #     return zeros(eltype(Z), 0)
    # end
    
    # initial guess 
    Xref, Uref = reference_trajectory(params, xic, xg)
    z0 = zeros(idx.nz)
    for i=1:(N-1)
        z0[idx.x[i]] = Xref[i]
        z0[idx.u[i]] = Uref[i]
    end
    z0[idx.x[N]] = Xref[N]
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    
    Z = fmincon(quadracat_cost,quadracat_equality_constraint,quadracat_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, t_vec, params 
end

solve_quadracat_jump (generic function with 1 method)

In [ ]:
X, U, t_vec, params_dircol = solve_quadracat_jump(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------


In [ ]:
## Do stuff here...
## Finally, simulate forward pass to update state of robot in X

In [ ]:
# ------------plotting--------------
Xm = hcat(X...)
Um = hcat(U...)

plot(t_vec, Xm[3,:], xlabel="Time Step", ylabel="Base Height (m)", title="Jump Trajectory")

In [ ]:
qs = [xk[1:state_dim(model)÷2] for xk in X] # extract positions from state vector

mvis = initialize_visualizer(model)
xic = initial_state(model)
set_configuration!(mvis, xic[1:state_dim(model)÷2])
anim = MeshCat.Animation(mvis, t_vec, qs)
setanimation!(mvis, anim)
render(mvis)